# Thread level Persistence

#### **1. Checkpointers (in LangGraph)**

* Think of a **checkpointer** like a “save game” in a video game.
* It **remembers the state** of your workflow (like what data you’ve processed, what step you’re on).
* If something crashes or you want to resume later, the checkpointer lets you **continue from where you left off** instead of starting from scratch.

Example:
You’re processing 100 documents. After 50, your program stops.

* With a checkpointer → next time, it resumes at document 51.
* Without a checkpointer → it starts again from document 1.

---

#### **2. Threads (in LangGraph)**

* A **thread** is like a “conversation line” or “execution flow.”
* Each user session or workflow run can have its own **thread ID**.
* This keeps data and context separate between different tasks or users.

Example:

* User A’s chatbot session = Thread 1
* User B’s chatbot session = Thread 2
* They don’t mix, because each thread has its own memory and checkpoints.

---

#### **3. State Snapshot**

* A **snapshot** is like taking a photo of your program’s current state.
* It freezes everything (inputs, outputs, progress) so you can **restore it later**.
* Snapshots are created by checkpointers to know exactly where to restart.

Example: Chatbot saves a snapshot after every reply. If it crashes, it reloads the last snapshot and continues.

---

#### **4. Short-term Memory**

* Short-term memory = what the system remembers **within a single thread/session**.
* It’s like remembering the last few messages, but it doesn’t survive forever.

Example: The chatbot remembers what you said 2 minutes ago in the same session.

---

### **5. Long-term Memory**

* Long-term memory = what’s saved **permanently across sessions/threads**.
* It survives restarts and can be recalled even in a new conversation.

Example: The chatbot remembers your name and preferences even if you come back tomorrow.

---



# Short-term memory with the InMemorySaver

In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage, RemoveMessage
from langgraph.checkpoint.memory import InMemorySaver
import config

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=config.gemini_api_key,
    temperature=0,
    max_tokens=250,
    model_kwargs={"seed": 42}
)

In [4]:
def AskGeminiAI(question):
    response = llm.invoke(question)
    return response

In [5]:
class State(MessagesState):
    summary: str

In [6]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [14]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
        
    system_message = f'''
    Here's a quick summary of what's been discussed so far:
    {state.get("summary", "")}
    
    Keep this in mind as you answer the next question.
    '''
    
    response = AskGeminiAI([SystemMessage(system_message)] + state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

In [15]:
def summarize_messages(state: State) -> State:
    print(f"\n-------> ENTERING summarize_messages:")

    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
    
    summary_instructions = f'''
        Update the ongoing summary by incorporating the new lines of conversation below. 
        Build upon the previous summary rather than repeating it, 
        so that the result reflects the most recent context and developments.
        Respond only with the summary.
        
        Previous Summary:
        {state.get("summary", "")}
        
        New Conversation:
        {new_conversation}
        '''

    print(summary_instructions)

    summary = AskGeminiAI([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]

    return State(messages = remove_messages, summary = summary.content)

In [17]:
graph = StateGraph(State)

graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("summarize_messages", summarize_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "summarize_messages")
graph.add_edge("summarize_messages", END)

In [18]:
checkpointer = InMemorySaver()
graph_compiled = graph.compile(checkpointer)

In [19]:
print(graph_compiled.get_graph().draw_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
   +--------------+    
   | ask_question |    
   +--------------+    
           *           
           *           
           *           
      +---------+      
      | chatbot |      
      +---------+      
           *           
           *           
           *           
+--------------------+ 
| summarize_messages | 
+--------------------+ 
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      


In [20]:
config1 = {"configurable": {"thread_id": "1"}}
config2 = {"configurable": {"thread_id": "2"}}

In [21]:
graph_compiled.invoke(State(), config2)


-------> ENTERING ask_question:
What is your question?


 Who is Ash Ketchum?



-------> ENTERING chatbot:
================================== Ai Message ==================================

Ash Ketchum is the main protagonist of the *Pokémon* anime series. He's a young boy from Pallet Town who dreams of becoming a Pokémon Master. He travels the world with his Pokémon, most notably Pikachu, battling Gym Leaders, participating in tournaments, and encountering various adventures. After 25 seasons, he finally achieved his goal of becoming a World Champion.

-------> ENTERING summarize_messages:

        Update the ongoing summary by incorporating the new lines of conversation below. 
        Build upon the previous summary rather than repeating it, 
        so that the result reflects the most recent context and developments.
        Respond only with the summary.

        Previous Summary:
        

        New Conversation:
        ai: What is your question?

human: Who is Ash Ketchum?

ai: Ash Ketchum is the main protagonist of the *Pokémon* anime series. He's a yo

{'messages': [],
 'summary': "Ash Ketchum is the main protagonist of the *Pokémon* anime series. He's a young boy from Pallet Town who dreams of becoming a Pokémon Master. He travels the world with his Pokémon, most notably Pikachu, battling Gym Leaders, participating in tournaments, and encountering various adventures. After 25 seasons, he finally achieved his goal of becoming a World Champion. The user inquired about Ash Ketchum, prompting the AI to provide this information."}